# Examples and Features of Envyron Representations

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from envyron.domains.cell import EnvironGrid

## EnvironField and DirectField

In [ ]:
from envyron.representations.field import EnvironField
from dftpy.field import DirectField

In [ ]:
at = np.eye(3)*3
nr = np.array([2, 2, 2])
minimal_cell = EnvironGrid(at, nr)

In [ ]:
uniform_density = EnvironField(minimal_cell, data=np.ones(nr)*2, label='uniform')

An `EnvironField` is a child of a `DirectField`, which is a modified numpy array. It has the same properties of arrays, including a shape and an array representation

In [ ]:
uniform_density.shape

In [ ]:
uniform_density

While a more detailed exploration of the components (attributes and methods) of `DirectField` objects is reported in the following, the main characteristics of this class is to contain pointers to three other special classes that are instrumental to most of the calculations for periodic systems: 
* grid/cell: the class that contains the information on the simulation cell and the numerical discretization of space into a structured grid
* fft: a link to the Numpy.fft module and a wrapper to the same that also exploits parallelization
* mp: an interface to MPI parallelization

#### Components of DirectField and EnvironField

There are a total of 37 components of a `DirectField` object that are not shared with numpy arrays, plus 10 more components are generated by instantiating the variable. One additional method `EnvironField.standard_view()` was added in the Environ child class. 

In [ ]:
listarray=dir(np.ndarray)
listdirectfield=dir(DirectField)
newcomponents=[]
for i in listdirectfield:
    if i not in listarray: 
        newcomponents.append(i)

In [ ]:
print(len(newcomponents))
print(newcomponents)

In [ ]:
print(uniform_density.spl_coeffs)

In [ ]:
listuniformdensity=dir(uniform_density)
for i in listuniformdensity:
    if i not in listdirectfield:
        print(i)

In [ ]:
help(uniform_density.fft_object)

In [ ]:
help(uniform_density.mp)

There are 2 hidden attributes (`DirectField.__module__` and `DirectField.__dict__`) that contain basic information on the instance of the class. One private method (`DirectField._DirectField_scatter`) probably related to data parallelization, and one private method (`DirectField._calc_spline()`) that creates a bunch of additional attributes. 

In [ ]:
uniform_density.__dict__

`DirectField.norm()` computes the normalization constant of the field by taking the square root of the integral of the function squared $\sqrt{\int_{cell} f(\mathbf{r})^2 \mathrm{d}\mathbf{r}}=\sqrt{\sum_{i\in grid}f_i^2 \Delta V}$

In [ ]:
uniform_density.norm() 

while `DirectField.N` (and its private `DirectField._N`) contains the linear norm, i.e., the integral of the field

In [ ]:
uniform_density.N

In [ ]:
uniform_density.integral()

NOTE: changing the density in any way does not automatically adjust the value of N

In [ ]:
uniform_density[1]=0.5
print(uniform_density.N,uniform_density._N)
print(uniform_density.norm())

`DirectField.normalize()` generates a new scaled density whose square norm is one

In [ ]:
normalized_density = uniform_density.normalize()
print(normalized_density)

In [ ]:
print(normalized_density.N,normalized_density._N)
print(normalized_density.norm(),normalized_density.integral())

In [ ]:
uniform_density

amax, amin, amean, asum seem to perform basic math operations on the values of the DirectField

In [ ]:
print(uniform_density.amax(),uniform_density.amin(),uniform_density.amean(),uniform_density.asum())

In [ ]:
uniform_density[1]=2

In addition, it contains a link to the grid (`EnvironGrid` or `DirectGrid` object) that contains information about direct lattice, reciprocal lattice, and gridpoints

In [ ]:
uniform_density.grid.lattice

In [ ]:
print(uniform_density.grid,type(uniform_density.grid))

Note that the `grid` component is only present in the instance of the object, while the class contains a link to an ASE `ase.cell.Cell` object

In [ ]:
print(uniform_density.cell,type(uniform_density.cell))

However, the `DirectGrid` and `EnvironGrid` classes also have an ASE `Cell` component, so it is not clear why we need the duplicate

In [ ]:
uniform_density.grid.cell

The `cplx` attribute is a Boolean variable that identifies complex-valued fields

In [ ]:
print(uniform_density.cplx,type(uniform_density.cplx))

`DirectField` objects also allow to perform fast Fourier transforms to reciprocal space through its `DirectGrid.fft()` method and to perform more advanced operations that involve FFTs, such as computing its gradient (4 methods: `numerically_smooth_gradient`, `gradient`, `standard_gradient`, `super_smooth_gradient`), `laplacian`, `divergence`, filter out high frequency components (`cut_highg`) etc. 

In [ ]:
uniform_density.fft()

In [ ]:
uniform_density.gradient()

In [ ]:
uniform_density.laplacian()

The integer attribute `DirectField.rank` is used to distinguish scalar fields from vector fields (gradients) and, in Environ, higher rank fields (hessians). 

In [ ]:
uniform_density.rank

`DirectField.read()` and `DirectField.write()` are IO methods, but we need to explore what formats are supported

In [ ]:
help(uniform_density.read)